In [7]:
import os
import pickle
import multiprocessing as mp
import math
import pandas as pd

In [8]:

base_paths = [
    '/home/combined_everything_FL/run_data/d_snn_atks_defs_surrs',
    '/home/combined_everything_FL/run_data/d_ipm_increased_scale',
    '/home/combined_everything_FL/run_data/d_ann_atks_defs',
    '/home/combined_everything_FL/run_data/d_decrease_gaussian_std'
]
file_paths = [os.path.join(base_path, f) for base_path in base_paths for f in os.listdir(base_path) if f.startswith('exp_')]
class ExpRecord:
    pass

In [9]:
exps = []
for path in file_paths:
    exp = pickle.load(open(path, 'rb'))
    if exp.attack['type'] == 'IPM' and exp.attack['params']['scale'] == 0.1:
        continue
    if exp.attack['type'] == 'GaussRandom' and ('std' not in exp.attack['params'] or exp.attack['params']['std'] != 1.0):
        continue
    # if exp.attack['type'] == 'GaussRandom':
    #     print(exp.attack)
    exps.append(exp)
print(len(exps))

1032


In [10]:
data = []
for exp in exps:
    if len(getattr(exp, 'test_accs', [])) == 0 or getattr(exp, 'checkpointed_epoch', None) != 2000:
        continue
    # Extract the attributes from each experiment
    row = {
        'model': exp.model,
        'data': exp.dataset,
        'surr': exp.snn_hyperparams['surrogate']['type'][:-4] if exp.model.startswith('snn') else 'None',
        'atk': str(exp.attack['type']),
        'def': exp.aggregator['type'],
        'best_acc': max([i[1] for i in exp.test_accs]),
    }

    if row['atk'] == 'SignFlip': 
        row['atk'] = 'S.F.'
    elif row['atk'] == 'LabelFlip': 
        row['atk'] = 'L.F.'
    elif row['atk'] == 'GaussRandom': 
        row['atk'] = 'G.R.'

    if row['def'] == 'SignGuard': 
        row['def'] = 'S.G.'
    elif row['def'] == 'CenterClipping':
        row['def'] = 'C.C.'
    elif row['def'] == 'NormClipping':
        row['def'] = 'N.C.'

    data.append(row)

# Create the DataFrame
df = pd.DataFrame(data)

# Display the first few rows
df.head()

,model,data,surr,atk,def,best_acc
0,snn_vgg9,CIFAR10,Rectangle,MinMax,DnC,0.7202
1,snn_vgg9,CIFAR100,Rectangle,MinMax,RFA,0.0191
2,snn_vgg9,CIFAR10,Gaussian,L.F.,Mean,0.4522
3,snn_vgg9,CIFAR100,Triangle,MinMax,DnC,0.4641
4,snn_vgg9,CIFAR10,Rectangle,MinMax,RFA,0.1146


In [11]:
no_atk = df[df['atk'] == 'None']

In [12]:
no_atk

,model,data,surr,atk,def,best_acc
49,snn_fc,MNIST,Gaussian,None,Mean,0.9712
97,snn_vgg9,CIFAR10,Rectangle,None,Mean,0.7210
121,snn_fc,MNIST,Triangle,None,Mean,0.9704
145,snn_vgg9,CIFAR100,Gaussian,None,Mean,0.3978
167,snn_vgg9,CIFAR100,Rectangle,None,Mean,0.3997
200,snn_vgg9,CIFAR10,Gaussian,None,Mean,0.7135
227,snn_fc,FMNIST,Rectangle,None,Mean,0.8667
239,snn_vgg9,CIFAR10,FastSigmoid,None,Mean,0.7146
248,snn_fc,FMNIST,Triangle,None,Mean,0.8623
279,snn_fc,FMNIST,Gaussian,None,Mean,0.8635


In [13]:
a = no_atk.groupby(['model', 'data', 'atk', 'def'], as_index=False)['best_acc'].mean()
a = a.rename(columns={'best_acc': 'baseline'})
grouped_no_atk = a

In [14]:
grouped_no_atk

,model,data,atk,def,baseline
0,ann_fc,FMNIST,None,Mean,0.84560
1,ann_fc,MNIST,None,Mean,0.96890
2,ann_vgg9,CIFAR10,None,Mean,0.77660
3,ann_vgg9,CIFAR100,None,Mean,0.44630
4,snn_fc,FMNIST,None,Mean,0.86482
5,snn_fc,MNIST,None,Mean,0.97064
6,snn_vgg9,CIFAR10,None,Mean,0.73600
7,snn_vgg9,CIFAR100,None,Mean,0.42030


In [15]:
atk_no_def = df[(df['atk'] != 'None') & (df['def'] == 'Mean')]

In [16]:
grouped_atk_no_def = atk_no_def.groupby(['model', 'data', 'atk', 'def'], as_index=False)['best_acc'].mean()
grouped_atk_no_def = grouped_atk_no_def.rename(columns={'best_acc': 'acc_atked'})

In [17]:
grouped_atk_no_def

,model,data,atk,def,acc_atked
0,ann_fc,FMNIST,Fang,Mean,0.81740
1,ann_fc,FMNIST,G.R.,Mean,0.40870
2,ann_fc,FMNIST,IPM,Mean,0.81390
3,ann_fc,FMNIST,L.F.,Mean,0.78760
4,ann_fc,FMNIST,Mimic,Mean,0.84120
5,ann_fc,FMNIST,MinMax,Mean,0.59870
6,ann_fc,FMNIST,S.F.,Mean,0.73670
7,ann_fc,MNIST,Fang,Mean,0.92380
8,ann_fc,MNIST,G.R.,Mean,0.75380
9,ann_fc,MNIST,IPM,Mean,0.94680


In [18]:
a = pd.merge(grouped_no_atk, grouped_atk_no_def, how='left', on=['model', 'data', 'def'])
a['rel_drop'] = (a['baseline'] - a['acc_atked'])/a['baseline']
a['rel_retention'] = a['acc_atked']/a['baseline']
a['drop'] = a['baseline'] - a['acc_atked']
a = a[['model', 'data', 'atk_y', 'baseline', 'acc_atked', 'rel_drop', 'rel_retention', 'drop']]
a = a.rename(columns={'atk_y': 'atk'})
atk_no_atk_cmp = a
atk_no_atk_cmp

,model,data,atk,baseline,acc_atked,rel_drop,rel_retention,drop
0,ann_fc,FMNIST,Fang,0.84560,0.81740,0.033349,0.966651,0.02820
1,ann_fc,FMNIST,G.R.,0.84560,0.40870,0.516675,0.483325,0.43690
2,ann_fc,FMNIST,IPM,0.84560,0.81390,0.037488,0.962512,0.03170
3,ann_fc,FMNIST,L.F.,0.84560,0.78760,0.068590,0.931410,0.05800
4,ann_fc,FMNIST,Mimic,0.84560,0.84120,0.005203,0.994797,0.00440
5,ann_fc,FMNIST,MinMax,0.84560,0.59870,0.291982,0.708018,0.24690
6,ann_fc,FMNIST,S.F.,0.84560,0.73670,0.128784,0.871216,0.10890
7,ann_fc,MNIST,Fang,0.96890,0.92380,0.046548,0.953452,0.04510
8,ann_fc,MNIST,G.R.,0.96890,0.75380,0.222004,0.777996,0.21510
9,ann_fc,MNIST,IPM,0.96890,0.94680,0.022809,0.977191,0.02210


In [19]:
a = atk_no_atk_cmp[atk_no_atk_cmp['model'].str.startswith('ann')]
b = atk_no_atk_cmp[atk_no_atk_cmp['model'].str.startswith('snn')]
a = a[['model', 'data', 'atk', 'rel_drop', 'rel_retention', 'drop']].rename(columns={'rel_drop': 'rel_drop_ann', 'rel_retention': 'retention_ann', 'drop': 'drop_ann'})
b = b[['model', 'data', 'atk', 'rel_drop', 'rel_retention', 'drop']].rename(columns={'rel_drop': 'rel_drop_snn', 'rel_retention': 'retention_snn', 'drop': 'drop_snn'})
c = pd.merge(a,b, how='left', on=['data', 'atk'])[['atk', 'data', 'drop_ann', 'drop_snn', 'rel_drop_ann', 'rel_drop_snn', 'retention_ann', 'retention_snn']]
c['snn_better'] = (c['drop_ann'] > c['drop_snn'])
c['snn_better(rel)'] = (c['retention_ann'] < c['retention_snn'])
c['snn_effective'] = c['drop_ann'] - c['drop_snn']
c.sort_values(['atk', 'data'])
cmp_ann_snn_atk = c

## Effectiveness of defenses on ANN vs SNN

In [20]:
df.head()

,model,data,surr,atk,def,best_acc
0,snn_vgg9,CIFAR10,Rectangle,MinMax,DnC,0.7202
1,snn_vgg9,CIFAR100,Rectangle,MinMax,RFA,0.0191
2,snn_vgg9,CIFAR10,Gaussian,L.F.,Mean,0.4522
3,snn_vgg9,CIFAR100,Triangle,MinMax,DnC,0.4641
4,snn_vgg9,CIFAR10,Rectangle,MinMax,RFA,0.1146


In [21]:
a = df
a = a[(a['atk'] != 'None') & (a['def'] != 'Mean')]
grouped_atk_def = a.groupby(['model', 'data', 'atk', 'def'], as_index=False)['best_acc'].mean().rename(columns={'best_acc': 'acc_defed'})
grouped_atk_def

,model,data,atk,def,acc_defed
0,ann_fc,FMNIST,Fang,C.C.,0.82730
1,ann_fc,FMNIST,Fang,DnC,0.84110
2,ann_fc,FMNIST,Fang,N.C.,0.84160
3,ann_fc,FMNIST,Fang,RFA,0.84490
4,ann_fc,FMNIST,Fang,S.G.,0.85100
...,...,...,...,...,...
275,snn_vgg9,CIFAR100,S.F.,C.C.,0.06956
276,snn_vgg9,CIFAR100,S.F.,DnC,0.30670
277,snn_vgg9,CIFAR100,S.F.,N.C.,0.09728
278,snn_vgg9,CIFAR100,S.F.,RFA,0.08912


In [22]:
a, b = grouped_atk_no_def, grouped_atk_def
c = grouped_no_atk[['model', 'data', 'baseline']]
d = pd.merge(a, b, 'left', ['model', 'data', 'atk'])
e = pd.merge(c, d, 'left', ['model', 'data'])
e['gain'] = (e['acc_defed'] - e['acc_atked'])/(e['baseline'] - e['acc_atked'])
e = e.rename(columns={'def_y': 'def'})[['model', 'data', 'atk', 'def', 'gain']]
# c['atk_def'] = c['atk'] + '_' + c['def']
# c['data_def'] = c['data'] + '_' + c['def']
def_gain = e
e

,model,data,atk,def,gain
0,ann_fc,FMNIST,Fang,C.C.,0.351063
1,ann_fc,FMNIST,Fang,DnC,0.840426
2,ann_fc,FMNIST,Fang,N.C.,0.858158
3,ann_fc,FMNIST,Fang,RFA,0.975177
4,ann_fc,FMNIST,Fang,S.G.,1.191490
...,...,...,...,...,...
275,snn_vgg9,CIFAR100,S.F.,C.C.,0.007864
276,snn_vgg9,CIFAR100,S.F.,DnC,0.678660
277,snn_vgg9,CIFAR100,S.F.,N.C.,0.086275
278,snn_vgg9,CIFAR100,S.F.,RFA,0.063193


In [29]:
a = def_gain
b, c = a[a['model'].str.startswith('ann')], a[a['model'].str.startswith('snn')]
d = pd.merge(b, c, 'left', ['data', 'atk', 'def'])
d['snn_advntg'] = d['gain_y'] - d['gain_x']
d['snn_better'] = d['snn_advntg'] > 0 
d = d[['data', 'atk', 'def', 'snn_advntg', 'snn_better']]
d['data_def'] = d['data'] + '_' + d['def']
d = d.sort_values(['data', 'def'])
e = d.pivot(
    index='atk',
    columns='data_def',
    values='snn_advntg'
)
e = e[[ 'MNIST_C.C.','MNIST_DnC','MNIST_N.C.','MNIST_RFA','MNIST_S.G.','FMNIST_C.C.','FMNIST_DnC','FMNIST_N.C.','FMNIST_RFA','FMNIST_S.G.','CIFAR10_C.C.','CIFAR10_DnC','CIFAR10_N.C.','CIFAR10_RFA','CIFAR10_S.G.','CIFAR100_C.C.','CIFAR100_DnC','CIFAR100_N.C.','CIFAR100_RFA','CIFAR100_S.G.']]
def_gain_comp = e
e

data_def,MNIST_C.C.,MNIST_DnC,MNIST_N.C.,MNIST_RFA,MNIST_S.G.,FMNIST_C.C.,FMNIST_DnC,FMNIST_N.C.,FMNIST_RFA,FMNIST_S.G.,CIFAR10_C.C.,CIFAR10_DnC,CIFAR10_N.C.,CIFAR10_RFA,CIFAR10_S.G.,CIFAR100_C.C.,CIFAR100_DnC,CIFAR100_N.C.,CIFAR100_RFA,CIFAR100_S.G.
atk,,,,,,,,,,,,,,,,,,,,
Fang,0.118455,0.000929,-0.878690,-0.225576,-0.099665,-0.327344,0.166215,-0.601042,-0.621287,-0.580484,0.131489,0.092196,0.301267,0.160383,-0.086002,0.528768,0.368909,0.524777,0.268835,-0.409369
G.R.,0.546417,-0.010489,-0.077767,-0.009676,-0.011209,-0.169994,-0.004040,-0.082559,-0.012218,-0.012561,-0.325891,0.001663,-0.024898,-0.013819,0.019668,-0.089542,0.011263,0.010483,-0.011619,-0.024880
IPM,-0.001964,-1.257723,0.055509,-32.714142,-15.260501,-6.066166,70.953027,-1.547170,658.010168,465.849494,0.075951,1.610872,35.770766,30.217169,-3.279329,-0.427473,-3.166783,27.040815,18.636985,-5.477541
L.F.,-0.149727,-0.032053,-0.960046,-0.033262,-0.194255,-0.299549,-0.022517,-0.509042,-0.027799,-0.383139,0.024658,-0.047469,-0.259692,-0.050999,-0.874955,0.048092,1.274065,-0.187971,0.013237,-0.173706
Mimic,-0.121956,-1.528464,0.260160,-15.904824,-4.630673,-0.626071,-0.025918,-1.014681,-0.407985,-0.541892,-1.435447,-4.112564,-3.462563,-81.042982,-8.281421,-0.205156,-0.466995,-0.045296,-10.473266,-0.041112
MinMax,-0.023547,-0.021267,-0.868495,-0.373357,-0.017556,1.852350,-0.005631,-0.128998,1.794087,-0.016076,-0.002286,0.003905,0.033925,0.006863,0.012820,0.003381,0.034567,0.008519,0.009798,0.007688
S.F.,-0.044752,0.086992,-0.303949,-0.017364,-0.255666,0.020661,0.023473,-0.354569,-0.075126,-0.241850,0.022891,0.018666,-0.199549,-0.375092,-0.238714,0.015573,0.627449,-0.067910,-0.062633,-0.274589


In [30]:
import re

def parse_csv_string(csv_string):
    """Parse a CSV string with a specific format where the first line contains column headers."""
    lines = csv_string.strip().split('\n')
    
    # Parse header line
    header_line = lines[0]
    columns = header_line.split(',')
    
    # Extract dataset and defense names
    datasets = []
    defenses = []
    for col in columns[1:]:  # Skip the first column (attack name)
        if re.match(r'[A-Za-z0-9]+_[A-Za-z0-9.]+', col):
            parts = col.split('_')
            dataset = parts[0]
            defense = parts[1]
            
            if dataset not in datasets:
                datasets.append(dataset)
            if defense not in defenses:
                defenses.append(defense)
    
    # Sort unique defenses to ensure consistent order
    defenses = list(dict.fromkeys(defenses))
    
    # Parse data lines
    data = []
    for line in lines[1:]:
        if not line.strip():
            continue
        values = line.split(',')
        attack_name = values[0]
        numeric_values = [float(v) for v in values[1:]]
        data.append([attack_name] + numeric_values)
    
    return columns, datasets, defenses, data

def generate_latex_table(csv_string, decimals=2):
    """Generate a LaTeX table from CSV data with specified number of decimal places."""
    columns, datasets, defenses, data = parse_csv_string(csv_string)
    
    # Start building the LaTeX code
    latex = [
        "\\documentclass{article}",
        "\\usepackage{booktabs}",
        "\\usepackage{multirow}",
        "\\usepackage{array}",
        "\\usepackage{colortbl}",
        "\\usepackage{xcolor}",
        "\\usepackage{pdflscape}",
        "\\usepackage{siunitx}",
        "",
        "\\begin{document}",
        "",
        "\\begin{landscape}",
        "\\begin{table}",
        "\\centering",
        "\\small",
        "\\setlength{\\tabcolsep}{3.5pt}",
        "\\caption{Attack Performance across Datasets and Defenses}",
        f"\\begin{{tabular}}{{l*{{{len(columns)-1}}}{{S[table-format=-1.{decimals}]}}}}",
        "\\toprule"
    ]
    
    # Add the dataset headers
    dataset_header = "\\multirow{2}{*}{atk}"
    for dataset in datasets:
        dataset_count = sum(1 for col in columns if col.startswith(f"{dataset}_"))
        dataset_header += f" & \\multicolumn{{{dataset_count}}}{{c}}{{{dataset}}}"
    latex.append(dataset_header + " \\\\")
    
    # Add the cmidrule separators
    cmidrules = []
    start_idx = 2
    for dataset in datasets:
        dataset_count = sum(1 for col in columns if col.startswith(f"{dataset}_"))
        end_idx = start_idx + dataset_count - 1
        cmidrules.append(f"\\cmidrule(lr){{{start_idx}-{end_idx}}}")
        start_idx = end_idx + 1
    latex.append(" ".join(cmidrules))
    
    # Add the defense headers
    defense_header = " "
    for col in columns[1:]:
        defense = col.split('_')[1]
        defense_header += f" & {{{defense}}}"
    latex.append(defense_header + " \\\\")
    latex.append("\\midrule")
    
    # Add the data rows
    for row in data:
        attack_name = row[0]
        values = [f"{val:.{decimals}f}" for val in row[1:]]
        latex.append(f"{attack_name} & " + " & ".join(values) + " \\\\")
    
    # Finish the table
    latex.extend([
        "\\bottomrule",
        "\\end{tabular}",
        "\\end{table}",
        "\\end{landscape}",
        "",
        "\\end{document}"
    ])
    
    return "\n".join(latex)


print(generate_latex_table(def_gain_comp.to_csv(index=True), decimals=2))


\documentclass{article}
\usepackage{booktabs}
\usepackage{multirow}
\usepackage{array}
\usepackage{colortbl}
\usepackage{xcolor}
\usepackage{pdflscape}
\usepackage{siunitx}

\begin{document}

\begin{landscape}
\begin{table}
\centering
\small
\setlength{\tabcolsep}{3.5pt}
\caption{Attack Performance across Datasets and Defenses}
\begin{tabular}{l*{20}{S[table-format=-1.2]}}
\toprule
\multirow{2}{*}{atk} & \multicolumn{5}{c}{MNIST} & \multicolumn{5}{c}{FMNIST} & \multicolumn{5}{c}{CIFAR10} & \multicolumn{5}{c}{CIFAR100} \\
\cmidrule(lr){2-6} \cmidrule(lr){7-11} \cmidrule(lr){12-16} \cmidrule(lr){17-21}
  & {C.C.} & {DnC} & {N.C.} & {RFA} & {S.G.} & {C.C.} & {DnC} & {N.C.} & {RFA} & {S.G.} & {C.C.} & {DnC} & {N.C.} & {RFA} & {S.G.} & {C.C.} & {DnC} & {N.C.} & {RFA} & {S.G.} \\
\midrule
Fang & 0.12 & 0.00 & -0.88 & -0.23 & -0.10 & -0.33 & 0.17 & -0.60 & -0.62 & -0.58 & 0.13 & 0.09 & 0.30 & 0.16 & -0.09 & 0.53 & 0.37 & 0.52 & 0.27 & -0.41 \\
G.R. & 0.55 & -0.01 & -0.08 & -0.01 & -0.01 & -0.